## Typy agentóþw w LangGraph – ReAct, Chain‑of‑Thought, Custom

### Instalacja bibliotek

In [1]:
%pip install -U langgraph langchain langchain-openai

  Using cached langchain_core-0.3.76-py3-none-any.whl.metadata (3.7 kB)
Using cached langchain_core-0.3.76-py3-none-any.whl (447 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 947.6/947.6 kB 10.0 MB/s  0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.102.0
    Uninstalling openai-1.102.0:
      Successfully uninstalled openai-1.102.0
  Attempting uninstall: langchain-core━━━━━━━━━━ 0/3 [openai]
    Found existing installation: langchain-core 0.3.750/3 [openai]
    Uninstalling langchain-core-0.3.75:━━━━━ 0/3 [openai]
      Successfully uninstalled langchain-core-0.3.752m0/3 [openai]
  Attempting uninstall: langchain-openai━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/3 [langchain-core]
    Found existing installation: langchain-openai 0.3.32━━━━━━ 1/3 [langchain-core]
    Uninstalling langchain-openai-0.3.32:━━━━━━━━━━━━━━━━━━━━━ 1/3 [langchain-core]
      Successfully uninstalled langchain-openai-0.3.32━━━━━━━━ 1/3 [langchain-core]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:

# Imports & config
import os
import operator
from typing import Annotated, List, TypedDict, Literal

from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END

from dotenv import load_dotenv
load_dotenv()

# You can override via environment: export OPENAI_MODEL="gpt-4o-mini" (or a reasoning model)
MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")
llm = ChatOpenAI(model=MODEL, temperature=0)


### ReAct Agent
ReAct Agent łączy rozumowanie krok po kroku z wykonywaniem akcji poprzez narzędzia, np. kalkulator czy wyszukiwarkę; świetny do dynamicznych problemów wymagających integracji z zewnętrznymi funkcjami.

In [3]:

from langgraph.prebuilt import create_react_agent

@tool
def calculator(expr: str) -> str:
    """Safely evaluate a basic arithmetic expression (digits and + - * / ( ) .)."""
    import math
    allowed = set("0123456789+-*/(). ")
    if any(c not in allowed for c in expr):
        raise ValueError("Only basic arithmetic characters are allowed.")
    # Provide a tiny globals dict (no builtins), and a math namespace with sqrt, etc.
    return str(eval(expr, {"__builtins__": {}}, {"sqrt": math.sqrt}))  # type: ignore[name-defined]

@tool
def lookup_country_capital(country: str) -> str:
    """Return a country's capital using a tiny built-in table."""
    capitals = {
        "poland": "Warsaw",
        "germany": "Berlin",
        "france": "Paris",
        "spain": "Madrid",
        "italy": "Rome",
    }
    return capitals.get(country.strip().lower(), "Unknown")

tools = [calculator, lookup_country_capital]

react_agent = create_react_agent(llm, tools)

# Try it!
prompt = (
    "Compute sqrt(144) with the calculator, then add the number of letters in the capital of Germany. "
    "Return only the final integer result."
)

result = react_agent.invoke({ "messages": [("user", prompt)] })
print(result["messages"][-1].content)


/tmp/ipykernel_14219/51448629.py:27: LangGraphDeprecatedSinceV10: create_react_agent has been moved to langchain.agents. Please update your import to 'from langchain.agents import create_react_agent'. Deprecated in LangGraph V1.0 to be removed in V2.0.
  react_agent = create_react_agent(llm, tools)


The final integer result is 18.


In [4]:

# (Optional) Streaming view – see intermediate steps as the agent reasons and calls tools
for event in react_agent.stream({ "messages": [("user", "What is 5*7 + sqrt(81)? Return only a number.")] }, stream_mode="values"):
    last = event["messages"][-1]
    role = getattr(last, "type", "assistant")
    print(f"[{role}] {last.content}")


[human] What is 5*7 + sqrt(81)? Return only a number.
[ai] 
[tool] Error: ValueError('Only basic arithmetic characters are allowed.')
 Please fix your mistakes.
[ai] 
[tool] 44
[ai] The result is 44.


### Chain‑of‑Thought
Chain-of-Thought Agent prowadzi wewnętrzne rozumowanie w kilku krokach (tzw. scratchpad), ale nie używa narzędzi; stosowany tam, gdzie wystarczy czysta analiza i dedukcja.

In [5]:

class CoTState(TypedDict):
    question: str
    # Accumulate steps across nodes if desired
    steps: Annotated[List[str], operator.add]
    answer: str

def plan_node(state: CoTState) -> dict:
    """Ask the LLM to propose a compact plan (steps) without solving yet."""
    sys = (
        "You are a careful planner. Break the user's question into 2-4 concise steps. "
        "Do not solve. Return only a numbered list of steps; no extra text."
    )
    messages = [("system", sys), ("user", state["question"])]
    resp = llm.invoke(messages)
    # Normalize into a list of strings (one per step)
    raw = resp.content
    steps = []
    for line in str(raw).splitlines():
        line = line.strip()
        if not line:
            continue
        # strip possible numbering like "1. ", "- ", etc.
        line = line.lstrip("-• ").split(". ", 1)[-1] if ". " in line[:4] else line.lstrip("-• ")
        steps.append(line)
    return {"steps": steps}

def solve_node(state: CoTState) -> dict:
    """Use the planned steps to derive the final answer only."""
    sys = (
        "Use the provided steps to solve the problem. "
        "Return only the final answer, no reasoning."
    )
    messages = [
        ("system", sys),
        ("user", f"Question: {state['question']}\nSteps: {state['steps']}"),
    ]
    resp = llm.invoke(messages)
    return {"answer": str(resp.content).strip()}

# Wire up the graph
graph = StateGraph(CoTState)
graph.add_node("plan", plan_node)
graph.add_node("solve", solve_node)
graph.add_edge(START, "plan")
graph.add_edge("plan", "solve")
graph.add_edge("solve", END)
cot_graph = graph.compile()


In [6]:
# Run the CoT graph
state = {
    "question": "If a book has 350 pages and I read 14 pages per day, how many days to finish?",
    "steps": [],
    "answer": ""
}
out = cot_graph.invoke(state)
print("Final answer:", out["answer"])

# If you're teaching and want to peek at the internal scratchpad (not for end users):
# print("Planned steps:", out["steps"])


Final answer: 25 days


### Custom Agent
Custom Agent daje pełną elstyczność, można samemu zdefiniować logikę, routing i węzły; używany, gdy żaden gotowy wzorzec nie pasuje do specyfiki zadania.

In [7]:

class CustomState(TypedDict):
    input: str
    task: Literal["math", "capitalize", "count"]
    result: str

def route(state: CustomState) -> str:
    """Deterministic router based on a simple protocol in the input."""
    text = state["input"].strip().lower()
    if text.startswith("math:"):
        return "math"
    if text.startswith("capitalize:"):
        return "capitalize"
    if text.startswith("count:"):
        return "count"
    # default fallback
    return "count"

def do_math(state: CustomState) -> dict:
    expr = state["input"].split(":", 1)[-1].strip()
    allowed = set("0123456789+-*/(). ")
    if any(c not in allowed for c in expr):
        return {"result": "Error: unsupported characters in math expression."}
    try:
        res = eval(expr, {"__builtins__": {}})
    except Exception as e:
        res = f"Error: {e}"
    return {"result": str(res)}

def do_capitalize(state: CustomState) -> dict:
    text = state["input"].split(":", 1)[-1].strip()
    return {"result": text.upper()}

def do_count(state: CustomState) -> dict:
    text = state["input"].split(":", 1)[-1].strip()
    tokens = [t for t in text.split() if t]
    return {"result": f"words={len(tokens)} chars={len(text)}"}

g = StateGraph(CustomState)
g.add_node("math", do_math)
g.add_node("capitalize", do_capitalize)
g.add_node("count", do_count)

# Conditional edges from router
g.add_conditional_edges(
    START,
    route,
    {
        "math": "math",
        "capitalize": "capitalize",
        "count": "count",
    },
)
g.add_edge("math", END)
g.add_edge("capitalize", END)
g.add_edge("count", END)

custom_agent = g.compile()


In [8]:

# Try several inputs
for user_input in [
    "math: (12 + 8) * 3",
    "capitalize: langgraph is great!",
    "count: How many words are here?",
]:
    out = custom_agent.invoke({"input": user_input, "task": "count", "result": ""})
    print(f"Input: {user_input}\nResult: {out['result']}\n---")


Input: math: (12 + 8) * 3
Result: 60
---
Input: capitalize: langgraph is great!
Result: LANGGRAPH IS GREAT!
---
Input: count: How many words are here?
Result: words=5 chars=24
---
